# オペレーティングシステム 演習 1
# スレッドプログラミングの基本

**このセルを編集して, 名前と学生証番号を書け.**

 * 名前 Name : 西山　晃人
 * 学生証番号 Student ID : 03-190503

**書けたら Shift + Enter で実行(入力を確定)させ, 保存(`Ctrl-S`)せよ**

# 1. この教材の使い方

1. 文章の部分を読んで概念を理解する
1. コードになっている部分は読んで理解し, `Shift + Enter` で実行して理解を確かめる
1. <font color="blue"><b>課題</b></font> と書かれている部分では指示に従ってコードを書いて, そのあとのコマンドが成功するようにする
1. 大概の場合セルに不完全なコードが提供されているのでそれを修正してプログラムを完成させよ
1. そのようなセルには "write your answer here" という指示がある. これは消さないこと 
1. 実行結果を確かめるセルには "Execute this cell and check the result" という指示がある. これも消さないこと

# <font color="blue">課題 1-1:</font>  練習用お試し課題

_以下のプログラムを修正し, hello world が3回出力されるようにせよ_

In [3]:
/* 以下の指示は消してはいけない(以降も同様) */
/* ------- このセルを修正して解答を書け. write your answer here ------- */

//% file: hello_world.c
//% cmd: gcc -O3 -Wall -Wextra -o hello_world hello_world.c

#include <stdio.h>
int main() {
  for(int i=0; i<3; i++){
    printf("hello world\n");
  }
  return 0;
}

_テスト用セル_

In [4]:
/* 以下の指示は消してはいけない(以降も同様) */
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

//% cmd: ./hello_world

hello world
hello world
hello world


# 2. スレッド (POSIXスレッド)

## 2-1. スレッドを使う一番単純なプログラム

* 二つのスレッドを作りそれぞれが3回helloと表示するだけのプログラム

## 利用API

* pthread_create(&tid,..,f,x) はf(x)を実行するスレッドを作る. スレッドのIDが変数tidに格納される
* pthread_join(tid,..) はスレッドtidの終了を待つ
* 特定の関数に関する情報(上記の..部分など)はググっても良いが, そのOS上の man コマンドで得るのがもっとも正確な上, コピペもしやすい(ググると高い確率でLinux用man pageが見つかるが, 知らないうちに他のOSのページを見ているかも知れない). 例:
```
$ man pthread_create
```
* 本演習環境はLinux (Ubuntu)なので手元のPCが学科PCの人は自分でやるのがよいし, そうでない人はクラウド環境のターミナルを使えば良い

In [5]:
//% file: hello_pthread.c
//% cmd: gcc -O3 -Wall -Wextra -o hello_pthread hello_pthread.c -lpthread

#include <stdio.h>
#include <pthread.h>
#include <unistd.h>

/* スレッドによって実行される関数 */
void * f(void * args_) {
  (void)args_;
  pthread_t tid = pthread_self();
  for (int i = 0; i < 3; i++) {
    printf("%ld : hello %d\n", tid, i);
    usleep(100);                /* 100 μ秒 = 0.0001 秒休眠 */
  }
  return 0;
}

int main() {
  pthread_t tid[2];
  /* スレッドを2個作る */
  for (int i = 0; i < 2; i++) {
    pthread_create(&tid[i], NULL, f, NULL);
  }
  /* 終了待ち */
  for (int i = 0; i < 2; i++) {
    pthread_join(tid[i], NULL);
  }
  return 0;
}


In [6]:
//% cmd: ./hello_pthread

140145598166784 : hello 0
140145589774080 : hello 0
140145598166784 : hello 1
140145589774080 : hello 1
140145598166784 : hello 2
140145589774080 : hello 2


## ポイント:

* スレッドは並行して動作するので両者が混ざりあった結果になる
* 1回表示する毎にusleepしているのは必須ではない
* usleepしなくても多数回表示しているうちに混ざり合うが, 3回程度ではあまりならないので, usleepすることで人工的に混ざりやすいようにしている

## 2-2. スレッドに引数を渡す

* すべての実用的なユースケースでは各スレッドに異なる動作をさせる
* そのためにpthread_createに異なる関数を渡すことも出来るが普通は, その関数に別の引数を与えて異なる動作をさせる
* 先程のプログラムを微修正して, fに異なる引数を与える例
* pthread_createに渡す関数はスレッドに複数の値を渡す, 返り値を受け取る, などからそのための構造体を作って

In [7]:
//% file: pthread_arg.c
//% cmd: gcc -O3 -Wall -Wextra -o pthread_arg pthread_arg.c -lpthread

#include <stdio.h>
#include <pthread.h>
#include <unistd.h>

/* スレッドに渡す情報を詰める構造体 */
typedef struct {
  pthread_t tid;
  long id;
  long n;                       /* 繰り返し数 */
  long slp_us;                     /* 繰り返し間のsleep時間 */
} thread_arg_t;

/* スレッドに実行される関数
   引数は void* 1つと決まっているが実際には thread_arg_tへのポインタ */
void * f(void * arg_) {
  thread_arg_t * arg = arg_;
  long id = arg->id;
  long n = arg->n;
  long slp_us = arg->slp_us;
  for (int i = 0; i < n; i++) {
    printf("%ld : hello %d\n", id, i);
    usleep(slp_us);                /* 100 μ秒 = 0.0001 秒休眠 */
  }
  return 0;
}

int main() {
  thread_arg_t args[2];
  /* スレッドを2個作る */
  for (int i = 0; i < 2; i++) {
    /* スレッドに渡す構造体に情報を詰める */
    args[i].id = i;
    args[i].n = (i + 1) * 5;
    args[i].slp_us = 1000 / args[i].n;
    pthread_create(&args[i].tid, NULL, f, &args[i]);
  }
  /* 終了待ち */
  for (int i = 0; i < 2; i++) {
    pthread_join(args[i].tid, NULL);
  }
  return 0;
}


In [8]:
//% cmd: ./pthread_arg

1 : hello 0
1 : hello 1
1 : hello 2
1 : hello 3
0 : hello 0
1 : hello 4
1 : hello 5
0 : hello 1
1 : hello 6
1 : hello 7
0 : hello 2
1 : hello 8
0 : hello 3
1 : hello 9
0 : hello 4


# 3. OpenMP

* pthread_createを使ってスレッドを作ると, いちいちそのための関数を定義してそれに引数を渡すための構造体を作ったりと, 決まりきったことのために手間がかかる
* OpenMPは並列処理のためのC/C++言語拡張で, スレッドを使った並列処理をするのであればこちらが簡単
* 例えば上記で作った pthread_arg.c と同じものは以下で済む

# 3-1. `#pragma omp parallel` ディレクティブ

In [9]:
//% file: hello_omp.c
//% cmd: gcc -fopenmp -o hello_omp hello_omp.c

#include <stdio.h>
#include <unistd.h>
#include <omp.h>

int main() {
  /* 以下の { ... } を実行するスレッドを2個作る */
#pragma omp parallel num_threads(2) 
  {
    long id = omp_get_thread_num(); /*  0 または 1 の ID */
    long n = (id + 1) * 5;
    long slp_us = 1000 / n;
    for (int i = 0; i < n; i++) {
      printf("%ld : hello %d\n", id, i);
      usleep(slp_us);                /* 100 μ秒 = 0.0001 秒休眠 */
    }
  }
  return 0;
}


In [10]:
//% cmd: ./hello_omp

0 : hello 0
1 : hello 0
1 : hello 1
0 : hello 1
1 : hello 2
1 : hello 3
0 : hello 2
1 : hello 4
0 : hello 3
1 : hello 5
1 : hello 6
0 : hello 4
1 : hello 7
1 : hello 8
1 : hello 9


* 一般に
```
#pragma omp parallel num_threads(n)
  文
```
で, 「文」を実行するスレッドをn個作る. num_threads(n)省略可能. 後にもう少し正確に述べる

* 注意として, n個のスレッドが実行するのは, #pragma omp parallel の「直下の一文」のみ. 以下では hello が n回, good bye は1回だけ表示される. 
```
#pragma omp parallel num_threads(n)
  printf("hello\n");
  printf("good bye\n");
```

* 複数の文を実行したければ複合文 { ... } にしたり, 関数呼び出しにすればよい. 

* #pragma omp parallel 全体はn個のスレッドすべてが「文」を実行し終えたら実行される. すなわち good bye は必ず hello すべての後に出力される

* 「文」中で
```
omp_get_thread_num()
```
は, これを呼び出したスレッドがnスレッド中の何番目のスレッドなのかの番号(0〜n-1)を返す. これを使って異なる挙動をさせることができる.
* 上記には出ていないが「文」中で
```
omp_get_num_threads()
```
は, 「文」を同時に実行しているスレッドの数(つまり上記のn)を返す. 

* OpenMPのスレッド = Pthreadのスレッドと思ってよく, したがってOpenMPのスレッドが普通にPthreadのプリミティブを利用可能

# 3-2. `#pragma omp for` ディレクティブ

* `#pragma omp parallel`によってnスレッドで実行されている文中に
```
#pragma omp for
  for 文
```
と書くと, for文の繰り返しをn個のスレッドで分け合って実行する. 

* 例えば以下は10回の繰り返しを2個のスレッドで分け合って実行する

In [11]:
//% file: hello_omp_for.c
//% cmd: gcc -fopenmp -o hello_omp_for hello_omp_for.c

#include <stdio.h>
#include <unistd.h>
#include <omp.h>

int main() {
#pragma omp parallel num_threads(2) 
  {
    int id = omp_get_thread_num();
    /* 以下のfor文の繰り返し(10回)が2スレッドで分割されて実行(つまり並列実行)される */
#pragma omp for
    for (int i = 0; i < 10; i++) {
      printf("thread %d executes iteration %d\n", id, i);
    }
  }
  return 0;
}


In [12]:
//% cmd: ./hello_omp_for

thread 0 executes iteration 0
thread 0 executes iteration 1
thread 0 executes iteration 2
thread 0 executes iteration 3
thread 0 executes iteration 4
thread 1 executes iteration 5
thread 1 executes iteration 6
thread 1 executes iteration 7
thread 1 executes iteration 8
thread 1 executes iteration 9


* `#pragma omp parallel` と`#pragma omp for` を組み合わせることでお手軽に for 文の並列処理ができる
* 注意: `#pragma omp parallel` はあくまで「全員が」同じ文を実行するもの, 言い換えれば「重複して実行する」文であったことに注意
* 以下は2 x 10 = 20行の出力がなされる

In [13]:
//% file: hello_omp_for_wrong.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o hello_omp_for_wrong hello_omp_for_wrong.c

#include <stdio.h>
#include <unistd.h>
#include <omp.h>

int main() {
  /* これはfor文の繰り返し全部(10回)をそれぞれのスレッドが実行するもの */
#pragma omp parallel num_threads(2) 
  {
    int id = omp_get_thread_num();
    for (int i = 0; i < 10; i++) {
      printf("thread %d executes iteration %d\n", id, i);
    }
  }
  return 0;
}


In [14]:
//% cmd: ./hello_omp_for_wrong

thread 1 executes iteration 0
thread 1 executes iteration 1
thread 1 executes iteration 2
thread 1 executes iteration 3
thread 1 executes iteration 4
thread 1 executes iteration 5
thread 1 executes iteration 6
thread 1 executes iteration 7
thread 1 executes iteration 8
thread 1 executes iteration 9
thread 0 executes iteration 0
thread 0 executes iteration 1
thread 0 executes iteration 2
thread 0 executes iteration 3
thread 0 executes iteration 4
thread 0 executes iteration 5
thread 0 executes iteration 6
thread 0 executes iteration 7
thread 0 executes iteration 8
thread 0 executes iteration 9


* `#pragma omp for` によって繰り返しがどのようにスレッドに分割されるかは色々と指定できるが深入りしない
 * 必要になったら, omp for schedule くらいでググってください

 * OpenMPや本格的な並列処理についてはまだ語るべきことがあるが, この演習では, 簡単にスレッドを何個か作ったり, 簡単にfor文を並列実行する手段として(のみ), OpenMPを使う. 

 * OpenMPの仕様の詳細については https://www.openmp.org/wp-content/uploads/OpenMP4.0.0.pdf を参照

# 3-3. `#pragma omp parallel` を実行するスレッド数

`#pragma omp parallel` の直後の文を実際にいくつのスレッドで実行するかについての規則をすべて書くとややこしい. この演習で必要な範囲に限って述べる

1. `omp_set_nested(1)`を実行しているかまたは環境変数`OMP_NESTED=true`が設定されている場合, 以下の3つのルールで定まる
 1. num_threads(n)が指定されていればn
 1. num_threadsが省略されており環境変数 OMP_NUM_THREADS=n が設定されていればn
 1. どちらでもない場合実装依存だが典型的にはその環境のプロセッサ数
1. `omp_set_nested(1)`を実行しておらず環境変数`OMP_NESTED=true`も設定されていない場合,
 1. その`#pragma omp parallel`がもっとも外側にある場合(つまりすでに実行中の`#pragma omp parallel`の中で(入れ子に)実行されているのではない場合), 上記と同じ
 1. そうでない場合1 (つまり, `#pragma omp parallel` が無視されたのと同じ状態)

以下では`omp_set_nested(1)`を実行しておくことで, fを実行するスレッドをm個, gを実行するスレッドをn個生成している

In [15]:
//% file: f_and_g.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o f_and_g f_and_g.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

void f(int nthreads) {
  /* nthreadsスレッドで実行 */
#pragma omp parallel num_threads(nthreads)
  {
    int id = omp_get_thread_num();
    int nt = omp_get_num_threads();
    printf("%d/%d: f\n", id, nt);
  }
}

void g(int nthreads) {
  /* nthreadsスレッドで実行 */
#pragma omp parallel num_threads(nthreads)
  {
    int id = omp_get_thread_num();
    int nt = omp_get_num_threads();
    printf("%d/%d: g\n", id, nt);
  }
}

int main(int argc, char ** argv) {
  int m = (argc > 1 ? atoi(argv[1]) : 2);
  int n = (argc > 2 ? atoi(argv[2]) : 3);
  /* omp_set_nested(1)を実行しているので, 以降は基本的に指定した数のスレッドが出来る */
  omp_set_nested(1);
  /* 全体としてfをmスレッド, gをnスレッドが実行 */
#pragma omp parallel num_threads(2) 
  {
    int id = omp_get_thread_num();
    if (id == 0) {
      f(m);
    } else if (id == 1) {
      g(n);
    }
  }
  return 0;
}


In [16]:
//% cmd: ./f_and_g 2 3

0/2: f
1/2: f
2/3: g
0/3: g
1/3: g


## 4 競合状態と同期

## 4-1. 競合状態の概念

* <font color="blue">競合状態 (race condition)</font>とは以下の状態のこと
 * 並行に実行されている複数のスレッドが同じ場所(変数や配列の要素等)をアクセスしている
 * そのうちの少なくとも一つは書き込みをしている
* 授業で説明したとおり, 競合状態のあるプログラムは大概の場合間違ったプログラムで, 結果が正しく計算されない, 毎回結果が違う, などの現象がおきうる
* 以下ではまずそれを体験する

* 以下は1つのスレッドがxを1000000回インクリメント (+1) するだけの単純なコード

In [17]:
//% file: update_x_1.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o update_x_1 update_x_1.c


#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

int main() {
  volatile long x = 0;
  for (long i = 0; i < 1000000; i++) {
    x++;
  }
  printf("x = %ld\n", x); fflush(stdout);
  return 0;
}


* 当然結果は1000000と表示される

In [18]:
//%cmd: ./update_x_1

x = 1000000


# <font color="blue"><b>課題1-2:</b></font> 競合状態を経験する

* _上記の for文を実行するスレッドを二つにして実行せよ_ 
* _2つのスレッドがそれぞれ x++ を 1000000回実行する. 合計で 2000000回実行される._
* ヒント: OpenMPのpragma omp parallelを使えば修正は1行だけ!

In [19]:
//% file: update_x_2.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o update_x_2 update_x_2.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

int main() {
  volatile long x = 0;
#pragma omp parallel num_threads(2) 
  for (long i = 0; i < 1000000; i++) {
    x++;
  }
  printf("x = %ld\n", x); fflush(stdout);
  return 0;
}


* 結果が 20000000 にはならないこと(偶然なる可能性もあるが), 実行する度に結果が違うことを, 以下を何度も実行して確認してみよ.

In [20]:
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */
//%cmd: ./update_x_2

x = 1289136


注:
 * ちなみにもちろんこの例は, xに10000000回, 1ずつ足すなどというおよそ意味のない計算で, だったらはじめからxに10000000を足せば良い. ただしそれにしても競合状態があることには変わりない. この例では競合状態が実際の間違いにつながる確率を高めるためにあえて1を10000000回足す, などということをしている
 * `volatile long x;` の `volatile` の意味を正確に説明するのは難しいが, 大雑把に言えば「xに対するアクセスは書かれた通りにやれ. 勝手に最適化して省略するな」という指示. これがないとコンパイラが勝手に上記のループを, `x += 10000000` と等価なものに変換する可能性もある. それをするなということ. これも誤動作の確率を高めるための(実用上ではなく, 教育上の)ミニ工夫. あってもなくても競合状態があることには代わりはない.

## 4-2. 同期の概念

* <font color="blue">「同期 (synchronization)」</font>という言葉は, 一般には「タイミングを揃える」という意味
* 並行プログラミングの文脈では, 放っておくと競合状態にあるプログラムにおいて, 二つの操作AとBが起きる<font color="blue">順番を制御</font>したり, AとBが<font color="blue">不可分(atomic)</font>に実行される(混ざりながら実行されない)ようにする, という意味で使われる
* 同期と言っても目的に応じていくつかの種類があるので以下では重要なものを演習する

## 4-3. 排他制御 (mutual exclusion)

* <font color="blue">排他制御</font>は「あるスレッドがAをしている間, 他のスレッドにはBをさせない」という目的にのために使われる
* 比喩は「一人しか入れない部屋」. 誰かが入っている間は他の人は入れない
* 英語はmutual exclusionで, 略して mutex (ミューテックス)と言うことがほとんど
* ロック(lock)と呼ばれることも多い. 部屋に入る=鍵をかける=lockする
* API (詳しくはman を読むこと)
 * `pthread_mutex_t` m; mutexのためのデータ構造
 * `pthread_mutex_init(&m, ...)`; 初期化
 * `pthread_mutex_lock(&m)`; lockをする(部屋に入る)
 * `pthread_mutex_unlock(&m)`; lockを開放する(部屋から出る)

# <font color="blue"><b>課題1-3:</b></font> 排他制御で不可分な更新を行う

* 前課題で作ったプログラムにmutexを導入して正しく動作する(つまりxが2000000になる)ようにせよ

In [21]:
//% file: update_x_3.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o update_x_3 update_x_3.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

int main() {
  volatile long x = 0;
  pthread_mutex_t m;
  pthread_mutex_init(&m, NULL);
#pragma omp parallel num_threads(2) 
  for (long i = 0; i < 1000000; i++) {
    pthread_mutex_lock(&m);
    x++;
    pthread_mutex_unlock(&m);
  }
  printf("x = %ld\n", x); fflush(stdout);
  assert(x == 2 * 1000000);
  return 0;
}


In [22]:
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */
//% cmd: ./update_x_3

x = 2000000


## 同期の内包(encapsulation), スレッドセーフティの概念

* あるデータ(今の例ではx)を触るのに同期(今の例では排他制御)が必要という時, そのためのデータ(今の例ではm)を別の変数として導入してユーザがそれを使って同期をしなくてはいけないというのは面倒なことこの上ない
* データそのものと, それにアクセスするのに必要な同期のためのデータ構造をひとまとめにしたもの(構造体)を作り, 行う操作は関数呼び出しにして, 使う方が特別な注意をしなくても良いようにする---同期をデータ構造の中に内包(encapsulate)する---というのが基本である
* 特段難しいことではなく, 今の場合であればデータ(x)とmutex (m)をセットにした構造体を作れば良い. 名前は, counter_t とでもしておくのが適切だろう
* また, 必要な同期が内包されていて呼び出すだけで複数スレッドから使えるようになっているデータ構造や関数を, 「スレッドセーフ(thread safe)」である, と呼ぶ

In [23]:
//% file: counter.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o counter counter.c 

#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

/* counter_t : スレッドセーフなカウンタ(同期が内包されたデータ構造) */
typedef struct {
  pthread_mutex_t m[1];         /* mutexを内包 */
  volatile long x;
} counter_t;

/* counterの初期化 */
void counter_init(counter_t * c) {
  pthread_mutex_init(c->m, NULL);
  c->x = 0;
}

/* counterに+1 */
void counter_inc(counter_t * c) {
  pthread_mutex_lock(c->m);
  c->x++;
  pthread_mutex_unlock(c->m);
}

/* counterの値を取り出す */
long counter_get(counter_t * c) {
  return c->x;
}

int main() {
  counter_t c[1];
  counter_init(c);
#pragma omp parallel
  {
    for (long i = 0; i < 10000000; i++) {
      counter_inc(c);
    }
  }
  printf("x = %ld\n", counter_get(c));
  return 0;
}


In [24]:
//% cmd: OMP_NUM_THREADS=2 ./counter

x = 20000000


In [25]:
//% cmd: OMP_NUM_THREADS=10 ./counter

x = 100000000


プログラム全体は明らかに長くなっているが, main関数がきれいに書けている

## 4-4. 条件変数 (condition variables)

* <font color="blue">条件変数(condition variable)</font>は「ある条件が成り立つまでスレッドを待たせる」ための一般的な仕組み(一般的であるがゆえにはじめは概念を理解しづらいかも知れない)
* よくある状況は, AとBを「A -> Bの順序で実行したい」という状況. 排他制御は「AとBを同時に実行しない」というだけで特定の順序を保証することは出来なかった事に注意
 * 典型的に A がある変数への書き込みでBがそれを読む, という場合. 正しく値をを受け渡すために, Aが書き込みを終えてからBが読むことを保証したい
 * 逆のケースもある. Aが読み込みで, Bが書き込み. Aの読み出しが終わってからでないと, 同じ変数を別の目的に使えないので, Aが読み出した後Bが書き込むことを保証したい

* 以下はスレッド0がxに12345を書き込み, もう一つのスレッドがそれを読み出すプログラム
 * もちろんこのままでは正しく動かない
 * なおここでは間違いが起きやすいように, スレッド0が書き込む前に usleep(100) (100μ秒寝る)をしているがこれがなくても間違ったプログラムであることに変わりはない

In [26]:
//% file: race2.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o race2 race2.c

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

int main() {
  volatile long x = 0;
#pragma omp parallel num_threads(2)
  {
    int id = omp_get_thread_num();
    if (id == 0) {
      usleep(100);
      x = 12345;
    } else {
      printf("x = %ld\n", x); fflush(stdout);
      assert(x == 12345);
    }
  }
  return 0;
}



In [27]:
//% cmd: ./race2

x = 0


race2: race2.c:20: main: Assertion `x == 12345' failed.
[C kernel] command exited with code -6, subsequent commands will not be executed

* 条件変数の使い方: ある条件 COND が成り立つまで待ちたい時のパターンは以下

```
pthread_mutex_lock(m);
while (!COND)
  pthread_cond_wait(c, m);
pthread_mutex_unlock(m);
```

一見するとなぜこんなややこしいコードが必要なのかわかりにくい. 授業で説明している.

結局今のケースは以下のようになる.

In [28]:
//% file: hello_cond.c
//% cmd: gcc -fopenmp -o hello_cond hello_cond.c

#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

int main() {
  volatile long x = 0;
  pthread_mutex_t m[1];
  pthread_cond_t c[1];
  pthread_mutex_init(m, NULL);  /* mutex初期化 */
  pthread_cond_init(c, NULL);  /* 条件変数初期化 */
#pragma omp parallel num_threads(2)
  {
    int id = omp_get_thread_num();
    if (id == 0) {
      usleep(100);
      pthread_mutex_lock(m);
      x = 12345;
      pthread_cond_broadcast(c); /* スレッド1が寝てたら起こす */
      pthread_mutex_unlock(m);
    } else {
      pthread_mutex_lock(m);
      while (x == 0) {
        pthread_cond_wait(c, m); /* まだ書かれてなかったら寝る(mは解放する. 起きたときはまた取得している) */
      }
      pthread_mutex_unlock(m);
      printf("x = %ld\n", x);
      assert(x == 12345);
      printf("OK\n");
    }
  }
  return 0;
}


In [29]:
//% cmd: ./hello_cond

x = 12345
OK


# 同期を内包した構造への変更

* ここでもひとつの値xを正しく受け渡したいだけなのにm, cと余分な変数を作り, 見にくいコードを書く羽目になっているので同期を内包したプログラムへの変更を試みる
* ここで実現すべきは, 値の書き込み(put), 値の読み出し(get)という二つの操作が可能で, かつgetはputがなされるまでは値を読み出さない, というデータ構造と関数である
* 簡単のためputは一度しか行われないとする
* wo_var_t (write-once variable)と命名する

In [30]:
//% file: wo_var.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o wo_var wo_var.c

#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>
#include <pthread.h>

/* write-once 変数
   あるスレッドがwo_var_put(v, x);
   あるスレッドがwo_var_get(v);
   とすると確実にxが読み出される.
   つまりgetがput後に行われることを保証する */

typedef struct {
  long val;
  pthread_mutex_t m[1];
  pthread_cond_t c[1];
} wo_var_t;

/* 初期化 */
void wo_var_init(wo_var_t * v) {
  pthread_mutex_init(v->m, NULL);
  pthread_cond_init(v->c, NULL);
  v->val = 0;
}

/* v に x を書く */
void wo_var_put(wo_var_t * v, long x) {
  pthread_mutex_lock(v->m);
  v->val = x;
  pthread_cond_broadcast(v->c);
  pthread_mutex_unlock(v->m);
}

/* v を読み出す. ただし put がされるまで待つ */
long wo_var_get(wo_var_t * v) {
  pthread_mutex_lock(v->m);
  while (v->val == 0) {
    pthread_cond_wait(v->c, v->m);
  }
  pthread_mutex_unlock(v->m);
  return v->val;
}

int main() {
  wo_var_t v[1];
  wo_var_init(v);
#pragma omp parallel num_threads(2)
  {
    int id = omp_get_thread_num();
    if (id == 0) {
      usleep(100);
      wo_var_put(v, 12345);
    } else {
      long x = wo_var_get(v);
      printf("x = %ld\n", x);
      assert(x == 12345);
      printf("OK\n");
    }
  }
  return 0;
}


In [31]:
//% cmd: ./wo_var

x = 12345
OK
